# Batch Processing Example

In this example, we use the `micasense.imageset` class to load a set of directories of images into a list of `micasense.capture` objects, and we iterate over that list saving out each image as an aligned stack of images as separate bands in a single tiff file each. Next, we use the metadata from the original captures to write out a log file of the captures and their locations.  Finally, we use `exiftool` from the command line to inject that metadata into the processed images, allowing us to stitch those images using commercial software such as Pix4D or Agisoft.

Note: for this example to work, the images must have a valid RigRelatives tag. This requires RedEdge version of at least 3.4.0 or any version of Altum.  If your images don't meet that spec, you can also follow this support ticket to add the RigRelatives tag to them: https://support.micasense.com/hc/en-us/articles/360006368574-Modifying-older-collections-for-Pix4Dfields-support

In [1]:
%load_ext autoreload
%autoreload 2

## Load Images into ImageSet

In [2]:
from ipywidgets import FloatProgress, Layout
from IPython.display import display
import micasense.imageset as imageset
import micasense.capture as capture
import os, glob
import multiprocessing

panelNames = None
useDLS = True

imagePath = os.path.expanduser(os.path.join('/dm','micasense','0000SET'))
panelNames = glob.glob(os.path.join(imagePath,'000','IMG_0000_*.tif'))
panelCap = capture.Capture.from_filelist(panelNames)

outputPath = os.path.join(imagePath,'..','stacks')
thumbnailPath = os.path.join(outputPath, '..', 'thumbnails')

overwrite = False # usefult to set to false to continue interrupted processing
generateThumbnails = True

# Allow this code to align both radiance and reflectance images; bu excluding
# a definition for panelNames above, radiance images will be used
# For panel images, efforts will be made to automatically extract the panel information
# but if the panel/firmware is before Altum 1.3.5, RedEdge 5.1.7 the panel reflectance
# will need to be set in the panel_reflectance_by_band variable.
# Note: radiance images will not be used to properly create NDVI/NDRE images below.
if panelNames is not None:
    panelCap = capture.Capture.from_filelist(panelNames)
else:
    panelCap = None

if panelCap is not None:
    if panelCap.panel_albedo() is not None:
        panel_reflectance_by_band = panelCap.panel_albedo()
    else:
        panel_reflectance_by_band = [0.67, 0.69, 0.68, 0.61, 0.67] #RedEdge band_index order
    panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)    
    img_type = "reflectance"
else:
    if useDLS:
        img_type='reflectance'
    else:
        img_type = "radiance"

In [3]:
## This progress widget is used for display of the long-running process
f = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Loading")
display(f)
def update_f(val):
    if (val - f.value) > 0.005 or val == 1: #reduces cpu usage from updating the progressbar by 10x
        f.value=val

%time imgset = imageset.ImageSet.from_directory(imagePath, progress_callback=update_f)
update_f(1.0)

FloatProgress(value=0.0, description='Loading', layout=Layout(width='100%'), max=1.0)

CPU times: user 6.07 s, sys: 163 ms, total: 6.24 s
Wall time: 19.2 s


In [4]:
import math
import numpy as np
from mapboxgl.viz import *
from mapboxgl.utils import df_to_geojson, create_radius_stops, scale_between
from mapboxgl.utils import create_color_stops
import pandas as pd

data, columns = imgset.as_nested_lists()
df = pd.DataFrame.from_records(data, index='timestamp', columns=columns)

#Insert your mapbox token here
token = 'pk.eyJ1IjoibWljYXNlbnNlIiwiYSI6ImNqYWx5dWNteTJ3cWYzMnBicmZid3g2YzcifQ.Zrq9t7GYocBtBzYyT3P4sw'
color_property = 'dls-yaw'
num_color_classes = 8

min_val = df[color_property].min()
max_val = df[color_property].max()

import jenkspy
breaks = jenkspy.jenks_breaks(df[color_property], nb_class=num_color_classes)

color_stops = create_color_stops(breaks,colors='YlOrRd')
geojson_data = df_to_geojson(df,columns[3:],lat='latitude',lon='longitude')

viz = CircleViz(geojson_data, access_token=token, color_property=color_property,
                color_stops=color_stops,
                center=[df['longitude'].median(),df['latitude'].median()], 
                zoom=16, height='600px',
                style='mapbox://styles/mapbox/satellite-streets-v9')
viz.show()

/home/stoermer/miniconda3/envs/micasense/lib/python3.7/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Define which warp method to use
For newer data sets with RigRelatives tags (images captured with RedEdge version 3.4.0 or greater with a valid calibration load, see https://support.micasense.com/hc/en-us/articles/360005428953-Updating-RedEdge-for-Pix4Dfields), we can use the RigRelatives for a simple alignment.

For sets without those tags, or sets that require a RigRelatives optimization, we can go through the Alignment.ipynb notebook and get a set of `warp_matrices` that we can use here to align.

In [5]:
from numpy import array
from numpy import float32

# Set warp_matrices to none to align using RigRelatives
# Or
# Use the warp_matrices derived from the Alignment Tutorial for this RedEdge set without RigRelatives
warp_matrices = [array([[ 1.0010375e+00, -3.9708032e-03, -8.9111652e+00],
       [ 1.6603530e-03,  1.0004413e+00,  4.3942509e+01],
       [-4.3470865e-07, -2.2260615e-06,  1.0000000e+00]], dtype=float32), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32), array([[ 9.9335790e-01, -4.7343201e-03, -1.4426430e-01],
       [ 3.0201012e-03,  9.9259055e-01,  3.3995308e+01],
       [-7.0813246e-07, -1.2657903e-06,  1.0000000e+00]], dtype=float32), array([[ 9.9653125e-01, -2.1419618e-03, -4.2702127e+00],
       [ 7.9199986e-04,  9.9769008e-01,  5.2042053e+01],
       [-9.7712336e-07, -4.9935232e-07,  1.0000000e+00]], dtype=float32), array([[ 9.9848622e-01, -3.2853740e-03, -9.9952917e+00],
       [ 1.8229613e-03,  9.9786657e-01,  1.7976559e+01],
       [-3.2802632e-07, -1.0995809e-06,  1.0000000e+00]], dtype=float32), array([[ 6.44834781e-02,  2.69729233e-04,  1.44456978e+01],
       [-2.84955924e-04,  6.46989490e-02,  1.00155299e+01],
       [-1.43807777e-06,  8.37883807e-07,  1.00000000e+00]])]

## Align images and save each capture to a layered tiff file

In [6]:
import exiftool
import datetime
## This progress widget is used for display of the long-running process
f2 = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Saving")
display(f2)
def update_f2(val):
    f2.value=val

if not os.path.exists(outputPath):
    os.makedirs(outputPath)
if generateThumbnails and not os.path.exists(thumbnailPath):
    os.makedirs(thumbnailPath)

# Save out geojson data so we can open the image capture locations in our GIS
with open(os.path.join(outputPath,'imageSet.json'),'w') as f:
    f.write(str(geojson_data))
    
try:
    irradiance = panel_irradiance+[0]
except NameError:
    irradiance = None

start = datetime.datetime.now()
for i,capture in enumerate(imgset.captures):
    imageName = os.path.basename(capture.images[0].path.replace('_1', '_RGBNET').replace('.tif', ''))
    outputFilename = imageName+'.tif'
    thumbnailFilename = imageName+'.jpg'
    fullOutputPath = os.path.join(outputPath, outputFilename)
    fullThumbnailPath= os.path.join(thumbnailPath, thumbnailFilename)
    if (not os.path.exists(fullOutputPath)) or overwrite:
        if(len(capture.images) == len(imgset.captures[0].images)):
            capture.create_aligned_capture(irradiance_list=irradiance, warp_matrices=warp_matrices)
            capture.save_capture_as_stack(fullOutputPath)
            if generateThumbnails:
                capture.save_capture_as_rgb(fullThumbnailPath)
    capture.clear_image_data()
    update_f2(float(i)/float(len(imgset.captures)))
update_f2(1.0)
end = datetime.datetime.now()

print("Saving time: {}".format(end-start))
print("Alignment+Saving rate: {:.2f} images per second".format(float(len(imgset.captures))/float((end-start).total_seconds())))

FloatProgress(value=0.0, description='Saving', layout=Layout(width='100%'), max=1.0)

Saving time: 0:04:48.690967
Alignment+Saving rate: 0.73 images per second


## Use Exiftool from the command line to write metadata to images

In [ ]:
import subprocess

## Preserves important exif information such as GPS, DateTimeOriginal, Focal Length, Camera Make, Camera Model
old_dir = os.getcwd()
os.chdir(outputPath)

for i,capture in enumerate(imgset.captures):
    raw_image = capture.images[0].path
    stacked_image = os.path.join(outputPath, os.path.basename(capture.images[0].path.replace('_1', '_RGBNET')))
    print("Write exif: {}".format(stacked_image))

    try:
        subprocess.call(['exiftool','-m','-ifd1:all=','-overwrite_original','-TagsFromFile',raw_image,stacked_image])
    finally:
        os.chdir(old_dir)

Write exif: /dm/micasense/0000SET/../stacks/IMG_0000_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0001_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0002_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0003_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0004_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0005_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0006_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0007_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0008_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0009_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0010_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0011_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0012_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0013_RGBNET.tif
Write exif: /dm/micasense/0000SET/../stacks/IMG_0014_RGBNET.tif
Write exif: /dm/micasense/0000SET/../sta